# Introduction


## Objective

Use Llama3 Langchain and ChromaDB to create a Retrieval Augmented Generation (RAG) system.

## Details

* **Model**: Llama 3  
* **Variation**: 8b-chat-hf  (8b: 8B dimm.; hf: HuggingFace)
* **Version**: V1  
* **Framework**: Transformers  




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.5/426.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.0 MB/s eta 

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.2 MB/s eta 0:00:00


In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

# Initialize model, tokenizer, query pipeline

In [ ]:
model_id = 'meta-llama/Meta-Llama-3-8B-Instruct'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

print(device)

cuda:0


In [ ]:
from huggingface_hub.hf_api import HfFolder

HfFolder.save_token('HF_TOKEN')

Prepare the model and the tokenizer.

In [ ]:
time_start = time()
model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_end = time()
print(f"Prepare model, tokenizer: {round(time_end-time_start, 3)} sec.")

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Prepare model, tokenizer: 242.081 sec.


Define the query pipeline.

In [ ]:
time_start = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        max_length=1024,
        device_map="auto",)
time_end = time()
print(f"Prepare pipeline: {round(time_end-time_start, 3)} sec.")

Prepare pipeline: 1.144 sec.


We define a function for testing the pipeline.

In [ ]:
def test_model(tokenizer, pipeline, message):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        message: the prompt
    Returns
        None
    """
    time_start = time()
    sequences = pipeline(
        message,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."

    question = sequences[0]['generated_text'][:len(message)]
    answer = sequences[0]['generated_text'][len(message):]

    return f"Question: {question}\nAnswer: {answer}\nTotal time: {total_time}"


## Test the query pipeline

In [ ]:
from IPython.display import display, Markdown
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [ ]:
response = test_model(tokenizer,
                    query_pipeline,
                   "Please explain what is EU AI Act.")
display(Markdown(colorize_text(response)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




**<font color='red'>Question:</font>** Please explain what is EU AI Act.


**<font color='green'>Answer:</font>**  The EU AI Act, also known as the Artificial Intelligence Act, is a proposed regulation that aims to regulate the development, deployment, and use of artificial intelligence (AI) in the European Union (EU). The act is designed to ensure that AI systems are safe, transparent, and fair, and to prevent their use for harmful purposes.
The EU AI Act is based on the idea that AI systems should be designed and developed with ethical considerations in mind, and that they should be held to high standards of transparency, accountability, and safety. The act would require AI developers to comply with certain standards and guidelines, such as:
1. Transparency: AI systems must be transparent about their decision-making processes and the data they use.
2. Accountability: AI systems must be designed to be accountable for their actions and decisions.
3. Safety: AI systems must be designed to ensure safety and prevent harm to individuals or society.
4. Fairness: AI systems must be designed


**<font color='magenta'>Total time:</font>** 24.025 sec.

In [ ]:
response = test_model(tokenizer,
                    query_pipeline,
                   "In the context of EU AI Act, how is performed the testing of high-risk AI systems in real world conditions?")
display(Markdown(colorize_text(response)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




**<font color='red'>Question:</font>** In the context of EU AI Act, how is performed the testing of high-risk AI systems in real world conditions?


**<font color='green'>Answer:</font>**  What are the key aspects that need to be considered for the testing of high-risk AI systems in real-world conditions? How does the EU AI Act address the testing of high-risk AI systems in real-world conditions? What are the potential challenges and limitations of the testing of high-risk AI systems in real-world conditions? How can the testing of high-risk AI systems in real-world conditions be improved? What are the potential benefits of the testing of high-risk AI systems in real-world conditions? How can the testing of high-risk AI systems in real-world conditions be integrated into the development process of AI systems? What are the key aspects that need to be considered for the integration of the testing of high-risk AI systems in real-world conditions into the development process of AI systems? How does the EU AI Act address the integration of the testing of high-risk AI systems in real-world conditions into the development process


**<font color='magenta'>Total time:</font>** 16.442 sec.

# Retrieval Augmented Generation

## Check the model with a HuggingFace pipeline

In [ ]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

# checking again that everything is working fine
time_start = time()
question = "Please explain what EU AI Act is."
response = llm(prompt=question)
time_end = time()
total_time = f"{round(time_end-time_start, 3)} sec."
full_response =  f"Question: {question}\nAnswer: {response}\nTotal time: {total_time}"
display(Markdown(colorize_text(full_response)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




**<font color='red'>Question:</font>** Please explain what EU AI Act is.


**<font color='green'>Answer:</font>**  What are the key provisions and implications for businesses and individuals? The EU AI Act is a proposed legislation aimed at regulating the development, deployment, and use of artificial intelligence (AI) within the European Union. The key provisions of the EU AI Act include:

1. Definition of AI: The act defines AI as a system that uses machine learning, deep learning, or other forms of AI to process data, make decisions, or perform tasks.
2. Risk assessment: The act requires AI developers to conduct a risk assessment before deploying an AI system to identify potential risks and take measures to mitigate them.
3. Transparency: The act requires AI developers to provide clear and transparent information about the AI system's capabilities, limitations, and potential biases.
4. Explainability: The act requires AI developers to provide explanations for the AI system's decisions and actions, particularly in cases where the AI system is used in critical decision-making or high-stakes applications.
5. Human oversight: The act requires AI developers to ensure that AI systems are designed and deployed with human oversight and control, particularly in cases where the AI system is used in high-stakes applications.
6. Data protection: The act requires AI developers to ensure that AI systems comply with EU data protection regulations, including the General Data Protection Regulation (GDPR).
7. Liability: The act establishes liability for AI developers and users in cases where AI systems cause harm or damage.
8. Certification: The act requires AI developers to obtain certification from an independent third-party auditor before deploying an AI system.

The implications of the EU AI Act for businesses and individuals are significant. The act aims to promote the development and deployment of trustworthy AI systems that are transparent, explainable, and safe. The act also aims to ensure that AI systems comply with EU data protection regulations and establish liability for AI developers and users.

For businesses, the EU AI Act means that they will need to:

1. Conduct risk assessments before deploying AI systems.
2. Provide clear and transparent information about AI systems.
3. Ensure that AI systems are designed and deployed with human oversight and control.
4. Comply with EU data protection regulations.
5. Obtain certification from an independent third-party auditor.

For individuals, the EU AI Act means that they will have the right to:

1. Understand how AI systems make decisions and take actions.
2. Request explanations for AI system decisions and actions.
3. Hold AI developers and users accountable for any harm or damage caused by AI systems.
4. Have their personal data protected and respected by AI systems.

Overall, the EU AI Act aims to promote the development and deployment of trustworthy AI systems that are transparent, explainable, and safe. The act also aims to ensure that AI systems comply with EU data protection regulations and establish liability for AI developers and users. – Source: EU AI Act Proposal (2020) and EU AI Act Explanation (2020) by the European Commission. – [1] [2] [3]

References:

[1] European Commission. (2020). AI Act Proposal. Retrieved from <https://ec.europa.eu/digital-single-market/en/artificial-intelligence-ai-act>

[2] European Commission. (2020). AI Act Explanation. Retrieved from <https://ec.europa.eu/digital-single-market/en/artificial-intelligence-ai-act-explained>

[3] European Commission. (2020). AI Act FAQs. Retrieved from <https://ec.europa.eu/digital-single-market/en/artificial-intelligence-ai-act-frequently-asked-questions> – [4] [5] [6]

References:

[4] European Union. (2020). Artificial Intelligence Act. Retrieved from <https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:52020PC0142&from=en>

[5] European Union. (2020). AI Act: Questions and Answers. Retrieved from <https://ec.europa.eu/digital-single-market/en/artificial-intelligence-ai-act-questions-and-answers>

[6] European Union. (2020). AI Act: FAQs. Retrieved from <https://ec.europa.eu/digital-single-market/en/artificial-intelligence-ai-act-frequently-asked-questions>

References:

[7] European Union. (2020). AI Act: Implementation. Retrieved from <https://ec.europa.eu/digital-single-market/en/artificial-intelligence-ai-act-implementation>

[8] European Union. (2020). AI Act: National Implementation. Retrieved from <https://ec.europa.eu/digital-single-market/en/artificial-intelligence-ai-act-national-implementation>

References:

[9] European Union. (2020). AI Act: EU-Wide Implementation. Retrieved from <https://ec.europa.eu/digital-single-market/en/artificial-intelligence-ai-act-eu-wide-implementation>

[10] European Union. (2020). AI Act: International Cooperation


**<font color='magenta'>Total time:</font>** 94.486 sec.

## Ingestion of data using Text loder

In [ ]:
loader = PyPDFLoader("/content/drive/MyDrive/GenAI_And_Applications_Course/eu_ai_act.pdf")
documents = loader.load()

## Split data in chunks using a recursive character text splitter.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)

## Creating Embeddings and Storing in Vector Store

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

## Initialize chain

In [ ]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

## Test the Retrieval-Augmented Generation

In [ ]:
def test_rag(qa, query):

    time_start = time()
    response = qa.run(query)
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."

    full_response =  f"Question: {query}\nAnswer: {response}\nTotal time: {total_time}"
    display(Markdown(colorize_text(full_response)))

In [ ]:
query = "How is performed the testing of high-risk AI systems in real world conditions?"
test_rag(qa, query)



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



> Finished chain.




**<font color='red'>Question:</font>** How is performed the testing of high-risk AI systems in real world conditions?


**<font color='green'>Answer:</font>**  According to Article 7, the testing of high-risk AI systems in real world conditions is performed at any point in time throughout the development process, and, in any event, prior to the placing on the market or the putting into service. The testing is made against prior defined metrics and is subject to a range of safeguards, including approval from the market surveillance authority, the right for affected persons to request data deletion, and the right for market surveillance authorities to request information related to testing. Additionally, the testing is without prejudice to ethical review that may be required by national or Union law. The testing plan must be submitted to the market surveillance authority in the Member State(s) where the testing is to be conducted. The testing is performed by the provider or prospective provider, either alone or in partnership with one or more prospective deployers. The testing is done in accordance with Article 54a and 54b. The testing is also subject to the requirements set out in this Chapter. The testing is done to ensure that the high-risk AI systems perform consistently for their intended purpose and are in compliance with the requirements set out in this Chapter. The testing is also done to identify the most appropriate and targeted risk management measures. The testing is done to ensure that the high-risk AI systems are in compliance with the requirements set out in this Chapter. The testing is done to ensure that the high-risk AI systems perform consistently for their intended purpose. The testing is done to identify the most appropriate and targeted risk management measures. The testing is done to ensure that the high


**<font color='magenta'>Total time:</font>** 28.514 sec.

In [ ]:
query = "What are the operational obligations of notified bodies?"
test_rag(qa, query)



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



> Finished chain.




**<font color='red'>Question:</font>** What are the operational obligations of notified bodies?


**<font color='green'>Answer:</font>**  According to Article 34a of the Regulation, the operational obligations of notified bodies include verifying the conformity of high-risk AI systems in accordance with the conformity assessment procedures referred to in Article 43. Notified bodies must also have documented procedures in place to safeguard impartiality and promote the principles of impartiality throughout their organisation, personnel, and assessment activities. Additionally, they must take full responsibility for the tasks performed by subcontractors or subsidiaries, and make a list of their subsidiaries publicly available. (Source: Regulation (EU) 2019/513)assistant:

The operational obligations of notified bodies, as stated in Article 34a of the Regulation, are:

1. Verifying the conformity of high-risk AI systems in accordance with the conformity assessment procedures referred to in Article 43.
2. Having documented procedures in place to safeguard impartiality and promote the principles of impartiality throughout their organisation, personnel, and assessment activities.
3. Taking full responsibility for the tasks performed by subcontractors or subsidiaries.
4. Making a list of their subsidiaries publicly available.

These obligations are intended to ensure that notified bodies operate in a transparent, impartial, and responsible manner, and that they maintain the trust and confidence of stakeholders in the conformity assessment process.assistant:

That's correct! Notified bodies play a crucial role in ensuring the conformity of


**<font color='magenta'>Total time:</font>** 25.071 sec.

## Document sources

In [ ]:
docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: What are the operational obligations of notified bodies?
Retrieved documents: 4
Source:  /kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf
Text:  5.
 
Notified bodies shall be organised and operated so as to safeguard the independence, 
objectivity and impartiality of their activities. Notified b
odies shall document and 
implement a structure and procedures to safeguard impartiality and to promote and apply 
the principles of impartiality throughout their organisation, personnel and assessment 
activities.
 
6.
 
Notified bodies shall have documented pro
cedures in place ensuring that their personnel, 
committees, subsidiaries, subcontractors and any associated body or personnel of external 

Source:  /kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf
Text:  authority accordingly.
 
2.
 
Notified bodies
 
shall take full responsibility for the tasks performed by subcontractors or 
subsidiaries wherever these are established.
 
3.
 
Activities may be subcontra